In [1]:
%%capture
#!pip install wfdb
BASEPATH = '../'
import sys
sys.path.insert(0,'../py')
print(sys.path)
import numpy as np
import os

from utils import reset_seed, plot_spectogram, plot_loss, load_data
from utils import integer_mapping, mapping_AAMI, MAP_AAMI, INT_MAP_DICT
from data_handling import DataHandling
from data_preprocessing import DataPreprocessing
from report import Reports

reset_seed()

In [2]:
#Load Preprocessed data
x_train, y_train = load_data(file_path = '../data/DS1_features_train.dat')
x_train.shape, len(y_train)
#Load Preprocessed data
x_val, y_val = load_data(file_path = '../data/DS1_features_val.dat')
x_val.shape, len(y_val)

File loaded: ../data/DS1_features_train.dat
File loaded: ../data/DS1_features_val.dat


((29152, 7, 33), 29152)

In [3]:
dh = DataHandling()
rp1 = dh.report_stats_table([y_train, y_val], ['train','val'])
rp1

,N,L,R,j,e,V,E,A,S,a,J,F,f,/,Q
train,5000,5526,5292,5005,5005,5156,5032,5103,5001,5040,5016,5220,0,0,5004
val,2200,2368,2268,2205,2205,2210,2201,2430,2201,2220,2210,2232,0,0,2202


In [4]:
#binary N vs. V
from data_augmentation import slice_data

x_train_sliced, y_train_sliced = slice_data(x_train,y_train,['N','L','R','j','e'])

dh = DataHandling()
rp1 = dh.report_stats_table([y_train_sliced], ['y_train_sliced'])
rp1

25828


,N,L,R,j,e,V,E,A,S,a,J,F,f,/,Q
y_train_sliced,5000,5526,5292,5005,5005,0,0,0,0,0,0,0,0,0,0


In [5]:
x_train = x_train_sliced
y_train = y_train_sliced

In [6]:
y_train = integer_mapping(y_train, INT_MAP_DICT)
y_val = integer_mapping(y_val, INT_MAP_DICT)

In [7]:
x_train = x_train.reshape(x_train.shape[0],-1)
x_val = x_val.reshape(x_val.shape[0],-1)

In [8]:
x_train.shape, x_val.shape

((25828, 231), (29152, 231))

In [9]:
%%time
from sklearn import svm
oneclass = svm.OneClassSVM(kernel='rbf')
oneclass.fit(x_train)

CPU times: user 2min 26s, sys: 168 ms, total: 2min 26s
Wall time: 2min 27s


OneClassSVM()

# Test

In [10]:
#Load Preprocessed Test Set
features_test, y_test = load_data(file_path = '../data/DS2_features.dat')
x_test = features_test
y_true = y_test.copy()
features_test.shape, len(y_test)

x_test = x_test.reshape(x_test.shape[0],-1)
x_test.shape

File loaded: ../data/DS2_features.dat


(49693, 231)

In [11]:
%%time
y_pred = oneclass.predict(x_test)

CPU times: user 2min 46s, sys: 14 µs, total: 2min 46s
Wall time: 2min 46s


In [12]:
np.unique(y_pred)

array([-1,  1])

In [12]:
def binarize_lables(y,positive_lable,pos=1,neg=-1):
    #y a list of lables
    #positive_lable: positive class lable 
    new_y = [pos if item==positive_lable else neg for item in y]
    return new_y  

In [13]:
y_true = binarize_lables(y_true,'N',pos=1,neg=-1)

In [14]:
#y_pred = integer_mapping(y_pred, INT_MAP_DICT, inverse=True)

In [15]:
#dh = DataHandling()
#rp2 = dh.report_stats_table([y_pred, y_true], ['y_pred','y_true'])
#rp2

In [16]:
labels = [k for k,v in INT_MAP_DICT.items()]
report = Reports(y_true=y_true, y_pred=y_pred, labels=labels)
normalize = None
#report.plot_confusion_matrix(normalize=normalize, values_format='')
print(report.the_classification_report())
#report.metrics_table()


y_pred_AAMI = mapping_AAMI(y_pred, MAP_AAMI)
y_true_AAMI = mapping_AAMI(y_true, MAP_AAMI)
labels_AAMI = ['N','V','S','F','Q']
report = Reports(y_true=y_true_AAMI, y_pred=y_pred_AAMI, labels=labels_AAMI)
#report.plot_confusion_matrix(normalize=normalize, values_format='')
print(report.the_classification_report())
report.metrics_table()

              precision    recall  f1-score   support

          -1     0.3115    0.5855    0.4066     13265
           1     0.7779    0.5287    0.6295     36428

    accuracy                         0.5439     49693
   macro avg     0.5447    0.5571    0.5181     49693
weighted avg     0.6534    0.5439    0.5700     49693



KeyError: -1